In [1]:
import numpy as np
from pymatching import Matching
import matplotlib.pyplot as plt
from scipy import sparse, linalg
import CompassCodes as cc
import csv
import pandas as pd
import os
from datetime import datetime
import sys
import glob

In [2]:
err_type = {0:"X", 1:"Z", 2:"CORR_XZ", 3:"TOTAL"}
d = 3
l = 2

In [5]:
def depolarizing_err(p, H, eta=0.5):
    """Generates the error vector for one shot according to depolarizing noise model.
       Args:
       - p: Error probability.
       - num_qubits: Number of qubits.
       - eta: depolarizing channel bias. Recover unbiased depolarizing noise eta = 0.5. 
                Px, py, pz are determined according to 2D Compass Codes paper (2019) defn of eta
       
       Returns:
       - A list containing error vectors for no error, X, Z, and Y errors.
    """
    num_qubits = H.shape[1]
    # Error vectors for I, X, Z, and Y errors
    errors = np.zeros((2, num_qubits), dtype=int)

    # p = px + py + pz, px=py, eta = pz/(px + py)
    px = 0.5*p/(1+eta)
    pz = p*(eta/(1+eta))
    probs = [1 - p, px, pz, px]  # Probabilities for I, X, Z, and Y errors

    # Randomly choose error types for all qubits
    # np.random.seed(10)
    choices = np.random.choice(4, size=num_qubits, p=probs)
    # Assign errors based on the chosen types
    errors[0] = np.where((choices == 1) | (choices == 3), 1, 0)  # X or Y error
    errors[1] = np.where((choices == 2) | (choices == 3), 1, 0)  # Z or Y error
    return errors

In [8]:
compass_code = cc.CompassCode(d=d, l=l)
H_x, H_z = compass_code.H['X'], compass_code.H['Z']
log_x, log_z = compass_code.logicals['X'], compass_code.logicals['Z']
eta = 0.5
shots = 100
p = 0.1

In [28]:
M_z = Matching.from_check_matrix(H_z)
M_x = Matching.from_check_matrix(H_x)
    
    # Generate error vectors
err_vec = [depolarizing_err(p, H_x, eta=eta) for _ in range(shots)]
err_vec_x = np.array([err[0] for err in err_vec])
err_vec_z = np.array([err[1] for err in err_vec])

# Syndrome for Z errors and decoding
syndrome_z = err_vec_x @ H_z.T % 2
correction_x = M_z.decode_batch(syndrome_z)
num_errors_x = np.sum((correction_x + err_vec_x) @ log_z % 2)

# Syndrome for X errors and decoding
syndrome_x = err_vec_z @ H_x.T % 2
correction_z = M_x.decode_batch(syndrome_x)
num_errors_z = np.sum((correction_z + err_vec_z) @ log_x % 2)

print(sum((correction_x + err_vec_x) @ log_z % 2))
print(sum((correction_z + err_vec_z) @ log_x % 2))

print(sum(((correction_x + err_vec_x) @ log_z % 2)|((correction_z + err_vec_z) @ log_x % 2)))

4
5
8
